In [7]:
%load_ext autoreload
%autoreload 2
import plotly.graph_objects as go
import utils
import plots
import pandas as pd
import numpy as np
pd.set_option('display.max_row', 1000)
pd.set_option('display.max_columns', 50)
data = {
    'plots': {},
    'stats': {}
}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')

In [9]:
def transpose(df):
    df = df.groupby(['Country/Region']).sum()
    df = df.drop(['Lat', 'Long'], axis=1)
    return df.transpose()

In [10]:
confirmed = transpose(confirmed)
deaths = transpose(deaths)

In [ ]:
fig = plots.confirmed_cases_over_threshold(confirmed)
fig.show()
data['plots']['figure_1'] = fig.to_json()

In [ ]:
fig = plots.countries_deaths_over_threshold(deaths)
fig.show()
data['plots']['figure_2'] = fig.to_json()

In [ ]:
fig = plots.top_countries_deaths_over_threshold_and_aligned(deaths)
fig.show()
data['plots']['figure_3'] = fig.to_json()

In [ ]:
fig = plots.deaths_over_threshold_and_aligned(deaths)
fig.show()
data['plots']['figure_4'] = fig.to_json()

In [ ]:
fig = plots.aligned_on_lockdown(deaths)
fig.show()
data['plots']['figure_5'] = fig.to_json()

In [ ]:
fig = plots.deaths_area(deaths)
fig.show()
data['plots']['deaths_area'] = fig.to_json()

In [ ]:
fig = plots.deaths_pie_chart(deaths)
fig.show()
data['plots']['deaths_pie_chart'] = fig.to_json()

In [ ]:
fig = plots.daily_change(deaths, 3)
fig.show()
data['plots']['daily_change'] = fig.to_json()

In [ ]:
fig = plots.daily_change(deaths.loc[pd.to_datetime(deaths.index) >= pd.to_datetime('2020-03-01')], 1)
fig.show()
data['plots']['daily_change2'] = fig.to_json()

In [ ]:
fig = plots.animated_map(deaths, 'iso_alpha.csv')
fig.show()
data['plots']['change_map'] = fig.to_json()

In [ ]:
fig = plots.rate_vs_total(deaths, 10, 5, 'death', limit=40)
fig.show()
data['plots']['peak_deaths'] = fig.to_json()

In [13]:
  threshold = 10
  deaths_sorted = utils.sort_columns_on_row(deaths)
  deaths_merged = deaths_sorted[deaths_sorted.columns[0:threshold]]
  deaths_merged['Other'] = deaths_sorted[deaths_sorted.columns[threshold:]].sum(axis=1)
  deaths_merged = deaths_merged.iloc[-1]

<ipython-input-13-02c515178e67>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [33]:
fig = plots.current_distribution(deaths)
fig.show()

c:\Appl\code\coronastatus\analysis\analysis\plots.py:534: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
fig = plots.rate_vs_total(confirmed, 5, 5, 'positive', limit=20)
fig.show()

In [ ]:
import stats
data['stats'] = stats.calc_stats(deaths, confirmed)

In [ ]:
import json
with open('../../frontend/src/data/data.json', 'w') as f:
    json.dump(data, f)